In [6]:
import altair as alt
import numpy as np
import pandas as pd
from scipy.interpolate import PchipInterpolator


In [3]:
month = 1
url = f"https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/all/20220{month}?type=daily_treasury_yield_curve&field_tdr_date_value_month=20220{month}&page&_format=csv"

In [4]:
pd.read_csv(url)

,Date,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,01/31/2022,0.03,0.13,0.22,0.49,0.78,1.18,1.39,1.62,1.75,1.79,2.17,2.11
1,01/28/2022,0.04,0.10,0.19,0.43,0.75,1.15,1.38,1.61,1.74,1.78,2.14,2.07
2,01/27/2022,0.04,0.14,0.20,0.43,0.75,1.18,1.43,1.66,1.78,1.81,2.17,2.09
3,01/26/2022,0.06,0.11,0.19,0.40,0.70,1.13,1.39,1.66,1.81,1.85,2.22,2.16
4,01/25/2022,0.05,0.09,0.19,0.39,0.65,1.02,1.28,1.56,1.73,1.78,2.18,2.12
5,01/24/2022,0.05,0.09,0.19,0.39,0.58,0.99,1.25,1.53,1.69,1.75,2.15,2.10
6,01/21/2022,0.05,0.08,0.17,0.35,0.58,1.01,1.28,1.54,1.70,1.75,2.13,2.07
7,01/20/2022,0.05,0.09,0.17,0.36,0.60,1.08,1.34,1.62,1.77,1.83,2.19,2.14
8,01/19/2022,0.05,0.06,0.17,0.36,0.57,1.04,1.33,1.62,1.78,1.83,2.20,2.14
9,01/18/2022,0.05,0.06,0.16,0.37,0.58,1.06,1.35,1.65,1.82,1.87,2.24,2.18


In [79]:
all_treasuries = pd.read_csv(
    "https://home.treasury.gov/system/files/276/yield-curve-rates-1990-2021.csv"
)

In [80]:
all_treasuries.Date = pd.to_datetime(all_treasuries.Date)


In [109]:
def interpolate_curve(curve):
    curve = curve.iloc[1:]
    curve.columns = ["tenor", "value"]
    curve.loc[:, "tenor"] = [1, 2, 3, 6, 12, 24, 36, 60, 90, 120, 240, 360]

    curve = curve.dropna()

    g = PchipInterpolator(curve["tenor"], curve["value"], extrapolate=True)
    tenors = np.linspace(0, 360, 61, endpoint=True)
    values = g(tenors)

    return tenors, values


In [110]:
old_treasury = all_treasuries.iloc[100].reset_index()
curve_date = all_treasuries.iloc[100].Date.strftime('%Y-%m-%d')


tenors, values = interpolate_curve(old_treasury)

curve_data = pd.DataFrame(
    {"tenor": np.linspace(0, 360, 61, endpoint=True), "value": values}
)

curve_data["yield"] = curve_data["yield"] / 100.0
curve_data["maturity"] = curve_data["maturity"] / 12


/var/folders/8g/67y55mms29zf6yt6q92_3hsr0000gn/T/ipykernel_57920/4098965440.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curve.loc[:, "tenor"] = [1, 2, 3, 6, 12, 24, 36, 60, 90, 120, 240, 360]


In [111]:
alt.Chart(curve_data).mark_line().encode(
    x="maturity", y="yield", tooltip=["maturity", "yield"]
)


alt.Chart(...)

In [ ]:
old_treasury = all_treasuries.iloc[100].reset_index()
curve_date = all_treasuries.iloc[100].Date.strftime('%Y-%m-%d')

tenors, values = interpolate_curve(old_treasury)

curve_data = pd.DataFrame(
    {"tenor": np.linspace(0, 360, 61, endpoint=True), "value": values}
)

curve_data["tenor"] = curve_data["tenor"] / 12
curve_data["value"] = curve_data["value"] / 100.0


In [113]:
curve_values = list(
    zip(
        np.tile("us_treasury", curve_data.shape[0]),
        np.tile(curve_date, curve_data.shape[0]),
        curve_data.maturity,
        curve_data["yield"],
    )
)


In [ ]:
for i in curve_values:
    query = """
        INSERT INTO curve
        (curve_name, date, tenor, value)
        VALUES (%s,%s,%s,%s)
        """
    